In [1]:
from harness import Harness

import sklearn.datasets
import sklearn.discriminant_analysis
import sklearn.mixture
import sklearn.model_selection

from pandas import ( 
    CategoricalIndex, DataFrame, Index, Series
)

from IPython.display import (
    Markdown,
)


In [2]:
"""Can I load the iris data"""
iris = sklearn.datasets.load_iris()

In [3]:
"""Can I initialize a Harness DataFrame"""
df = Harness(
    data=iris['data'], 
    index=CategoricalIndex(iris['target']).rename('target'),
    columns=iris['feature_names'],
    estimator=sklearn.discriminant_analysis.LinearDiscriminantAnalysis(),
    feature_level='target',
)

df = df.set_index(
    df.index
    .rename_categories(iris['target_names'])
    .rename('target_name'), append=True
).set_index(
    df
    .index.rename_categories(['red', 'green', 'blue'])
    .rename('color'), append=True
)

df.sample(5)

sepal length (cm)  sepal width (cm)  \
target target_name color                                        
1      versicolor  green                6.0               2.7   
0      setosa      red                  4.6               3.1   
                   red                  5.0               3.6   
1      versicolor  green                6.9               3.1   
                   green                4.9               2.4   

                          petal length (cm)  petal width (cm)  
target target_name color                                       
1      versicolor  green                5.1               1.6  
0      setosa      red                  1.5               0.2  
                   red                  1.4               0.2  
1      versicolor  green                4.9               1.5  
                   green                3.3               1.0

In [4]:
test_train = next(
    sklearn.model_selection
    .StratifiedKFold(n_splits=2)
    .split(df.values, df.Index('target'))
)

split = DataFrame(index=df.index, columns=['split'])
split.iloc[test_train[0]], split.iloc[test_train[1]] = 'train', 'test'
df = df.set_index(split.set_index('split', append=True).swaplevel(-1,0).index)

df.ix['train'].fit().sample(5)

sepal length (cm)  sepal width (cm)  \
target_name color target                                        
versicolor  green 1                     6.3               2.3   
                  1                     6.0               2.7   
virginica   blue  2                     5.9               3.0   
versicolor  green 1                     5.5               2.6   
virginica   blue  2                     7.2               3.2   

                          petal length (cm)  petal width (cm)  
target_name color target                                       
versicolor  green 1                     4.4               1.3  
                  1                     5.1               1.6  
virginica   blue  2                     5.1               1.8  
versicolor  green 1                     4.4               1.2  
virginica   blue  2                     6.0               1.8

In [5]:
df.ix['train'].score()

0.97333333333333338

In [6]:
df.add_template(
scoreboard="""
The table presents some information about the `{{
    df.estimator.__str__().split('(',1)[0]
}}` model applied to the `iris` dataset.

|         |           Test          |           Training       |
|---------|-------------------------|--------------------------|
| Score   |{{df.ix['test'].score()}}|{{df.ix['train'].score()}}|
| Samples | {{df.ix['test'] | len}} | {{df.ix['train'] | len}} |
"""
);

In [7]:
Markdown(df.get_template('scoreboard'))


The table presents some information about the `LinearDiscriminantAnalysis` model applied to the `iris` dataset.

|         |           Test          |           Training       |
|---------|-------------------------|--------------------------|
| Score   |0.986666666667|0.973333333333|
| Samples | 75 | 75 |

In [8]:
transformed = df.transform()
transformed.sample(2)

0         1
split target_name color target                    
test  setosa      red   0       7.341003 -0.791289
train virginica   blue  2      -5.517750  1.786963

In [9]:
with transformed.reset_index().DataSource(
    x=0, y=1, fill_color='color', text='target_name'
) as source:
    source.Scatter(color='text', marker='split')
    source.save()